In [1]:
from dask.distributed import LocalCluster, Client
import xarray as xr
import dask
import intake
import os
#import fsspec, os, netrc, aiohttp,dask
from satsearch import Search
import hvplot.pandas
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')

In [2]:
# NASA URS Authentication
import netrc, fsspec, aiohttp

(username, account, password) = netrc.netrc().authenticators("urs.earthdata.nasa.gov")
fsspec.config.conf['https'] = dict(client_kwargs={'auth': aiohttp.BasicAuth(username, password)})


In [3]:
#Setup GDAL Env for optimum performance
#env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
#           AWS_NO_SIGN_REQUEST='YES',
#           GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
#           GDAL_SWATH_SIZE='200000000',
#           VSI_CURL_CACHE_SIZE='200000000',
#           CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
#           GDAL_HTTP_UNSAFESSL='YES',
#           GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
#           GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))
#os.environ.update(env)

from osgeo import gdal
#gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS', 'tif')
gdal.SetConfigOption('VSI_CACHE', 'FALSE')
#gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN', 'YES')
#gdal.SetConfigOption('AWS_NO_SIGN_REQUEST', 'YES')
gdal.SetConfigOption('GDAL_MAX_RAW_BLOCK_CACHE_SIZE', '200000000')
gdal.SetConfigOption('GDAL_SWATH_SIZE', '200000000')
gdal.SetConfigOption('VSI_CURL_CACHE_SIZE', '200000000')
#gdal.SetConfigOption('GDAL_HTTP_UNSAFESSL', 'YES')
#gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE', os.path.expanduser('~\\cookies.txt'))
#gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', os.path.expanduser('~\\cookies.txt'))

gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE',os.path.expanduser('~\\cookies.txt'))
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', os.path.expanduser('~\\cookies.txt'))
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','EMPTY_DIR')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','.tif')

In [4]:
dask.config.set({'distributed.dashboard.link':'http://localhost:8888/proxy/8787/status'})#'https://localhost:8787/status'})
cluster = LocalCluster(threads_per_worker=2)
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:55888 Dashboard: http://localhost:8888/proxy/8787/status,Cluster Workers: 6 Cores: 12 Memory: 16.93 GB


In [5]:
def get_STAC_items(url, collection, dates, bbox):
    results = Search.search(url=url,
                        collections=[collection], 
                        datetime=dates,
                        bounding_box=bbox,    
                        sortby=['-properties.datetime'])

    items = results.items()
    print(f'Found {len(items)} Items')
    return intake.open_stac_item_collection(items)

In [6]:
url = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD' 
collection = 'C1711924822-LPCLOUD' #HLS
bbox=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
dates = '2013-01-01/2021-12-31'

cat = get_STAC_items(url,collection,dates,','.join(map(str, bbox)))

Found 38 Items


In [7]:
import numpy as np

In [8]:
cat.to_geopandas()

,geometry,datetime,start_datetime,end_datetime,eo:cloud_cover
0,"POLYGON ((-104.03568 40.55873, -103.68380 41.5...",2021-01-11T18:02:37.664Z,2021-01-11T18:02:37.664Z,2021-01-11T18:02:37.664Z,30
1,"POLYGON ((-103.70336 40.55547, -103.68380 41.5...",2021-01-08T17:52:41.637Z,2021-01-08T17:52:41.637Z,2021-01-08T17:52:41.637Z,8
2,"POLYGON ((-104.02647 40.55865, -103.68481 41.4...",2021-01-06T18:02:35.853Z,2021-01-06T18:02:35.853Z,2021-01-06T18:02:35.853Z,14
3,"POLYGON ((-103.70336 40.55547, -103.68380 41.5...",2021-01-03T17:52:39.618Z,2021-01-03T17:52:39.618Z,2021-01-03T17:52:39.618Z,16
4,"POLYGON ((-104.03284 40.55870, -103.68380 41.5...",2021-01-01T18:02:37.093Z,2021-01-01T18:02:37.093Z,2021-01-01T18:02:37.093Z,56
5,"POLYGON ((-103.70336 40.55547, -103.68380 41.5...",2020-12-29T17:52:40.885Z,2020-12-29T17:52:40.885Z,2020-12-29T17:52:40.885Z,67
6,"POLYGON ((-104.02009 40.55859, -103.68524 41.4...",2020-12-27T18:02:34.451Z,2020-12-27T18:02:34.451Z,2020-12-27T18:02:34.451Z,61
7,"POLYGON ((-103.70336 40.55547, -103.68380 41.5...",2020-12-24T17:52:37.970Z,2020-12-24T17:52:37.970Z,2020-12-24T17:52:37.970Z,26
8,"POLYGON ((-104.02717 40.55865, -103.68478 41.4...",2020-12-22T18:02:35.763Z,2020-12-22T18:02:35.763Z,2020-12-22T18:02:35.763Z,49
9,"POLYGON ((-103.70336 40.55547, -103.68380 41.5...",2020-12-19T17:52:39.337Z,2020-12-19T17:52:39.337Z,2020-12-19T17:52:39.337Z,68


In [14]:
import gdal
test = gdal.Open('/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020274T174141.v1.5.B01.tif')

In [16]:
fsspec.config.conf['https']

{'client_kwargs': {'auth': BasicAuth(login='spkearney', password='1mrChamu', encoding='latin1')}}

In [11]:
import intake
@dask.delayed
def lazy_open(href,chunks=dict(band=1, x=3660, y=3660)):
    sources = intake.open_rasterio(href, chunks=chunks, concat_dim='band')
    da = sources.to_dask() 
    return da

band = 'B01'
dataArrays = dask.compute(*[lazy_open(cat[item][band].urlpath) for item in list(cat)], retries=3)
da = xr.concat(dataArrays, dim='band', join='override', combine_attrs='drop').rename(band='time')
da['time'] = [cat[item].metadata['datetime'] for item in list(cat)]
da

RasterioIOError: '/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020294T174351.v1.5.B01.tif' not recognized as a supported file format.

In [9]:
da = da.persist()

In [15]:
da

<xarray.DataArray (time: 33, y: 3660, x: 3660)>
dask.array<concatenate, shape=(33, 3660, 3660), dtype=int16, chunksize=(1, 3660, 3660), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 2020-12-29T17:52:40.885000+00:00 ... 2020-07-10T18...
  * y        (y) float64 4.6e+06 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06 4.49e+06
  * x        (x) float64 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.097e+05 6.098e+05

In [23]:
da.isel(time=0).hvplot(x='x',y='y',rasterize=True,tiles='EsriImagery')

:DynamicMap   []
   :Overlay
      .Tiles.I :Tiles   [x,y]
      .Image.I :Image   [x,y]   (value)

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
from tqdm.notebook import tqdm

In [10]:
cat[list(cat)[0]]['B01'].to_dask()

<xarray.DataArray (band: 1, y: 3660, x: 3660)>
dask.array<open_rasterio-77818c6ac94556133020f01298a87dd0<this-array>, shape=(1, 3660, 3660), dtype=int16, chunksize=(1, 3660, 3660), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 4.6e+06 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06 4.49e+06
  * x        (x) float64 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.097e+05 6.098e+05
Attributes:
    transform:                                         (30.0, 0.0, 499980.0, ...
    crs:                                               +init=epsg:32613
    res:                                               (30.0, 30.0)
    is_tiled:                                          1
    nodatavals:                                        (-9999.0,)
    scales:                                            (0.0001,)
    offsets:                                           (0.0,)
    descriptions:                                      ('Coastal_Aerosol',)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    AREA_OR_POINT:                                     Area
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    arop_rmse(meters):                                 0
    arop_s2_refimg:                                    NONE
    cloud_coverage:                                    67
    DATASTRIP_ID:                                      S2A_OPER_MSI_L1C_DS_VG...
    HLS_PROCESSING_TIME:                               2020-12-31T16:22:02Z
    HORIZONTAL_CS_CODE:                                EPSG:32613
    HORIZONTAL_CS_NAME:                                WGS84 / UTM zone 13N
    L1C_IMAGE_QUALITY:                                 NONE
    L1_PROCESSING_TIME:                                2020-12-29T20:22:47.54...
    long_name:                                         Coastal_Aerosol
    MEAN_SUN_AZIMUTH_ANGLE:                            163.154684550781
    MEAN_SUN_ZENITH_ANGLE:                             66.0461164037811
    MEAN_VIEW_AZIMUTH_ANGLE:                           103.572543809972
    MEAN_VIEW_ZENITH_ANGLE:                            7.81728966000272
    MSI band 01 bandpass adjustment slope and offset:  0.995900, -0.000200
    MSI band 02 bandpass adjustment slope and offset:  0.977800, -0.004000
    MSI band 03 bandpass adjustment slope and offset:  1.005300, -0.000900
    MSI band 04 bandpass adjustment slope and offset:  0.976500, 0.000900
    MSI band 11 bandpass adjustment slope and offset:  0.998700, -0.001100
    MSI band 12 bandpass adjustment slope and offset:  1.003000, -0.001200
    MSI band 8a bandpass adjustment slope and offset:  0.998300, -0.000100
    NBAR_SOLAR_ZENITH:                                 66.4201760485218
    NCOLS:                                             3660
    NROWS:                                             3660
    OVR_RESAMPLING_ALG:                                NEAREST
    PROCESSING_BASELINE:                               02.09
    PRODUCT_URI:                                       S2A_MSIL1C_20201229T17...
    scale_factor:                                      0.0001
    SENSING_TIME:                                      2020-12-29T17:52:40.88...
    SPACECRAFT_NAME:                                   Sentinel-2A
    spatial_coverage:                                  94
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2A_OPER_MSI_L1C_TL_VG...
    ULX:                                               499980
    ULY:                                               4600020
    _FillValue:                                        -9999

datetime.datetime(2020, 12, 29, 17, 52, 40, 885000, tzinfo=tzlocal())

SyntaxError: invalid syntax (<ipython-input-34-7c92e4edb70b>, line 2)

In [14]:
v=[]
for item in tqdm(list(cat)):
    v.append(cat[item][band].to_dask(chunks='auto'))

NameError: name 'tqdm' is not defined

In [17]:
band = 'B01'
v = [cat[item][band].to_dask() for item in list(cat)]

application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/r

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [14]:
cat['G1948128420-LPCLOUD'].B01.to_dask()

/opt/conda/envs/py_geo/lib/python3.8/site-packages/intake_stac/catalog.py:456: UserWarning: STAC Asset "type" missing, assuming default type=application/rasterio:
{'name': 'Download HLS.S30.T13TEF.2020262T174919.v1.5.B8A.tif', 'href': 'https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020262T174919.v1.5.B8A.tif', 'type': 'application/rasterio'}
  warnings.warn(
/opt/conda/envs/py_geo/lib/python3.8/site-packages/intake_stac/catalog.py:456: UserWarning: STAC Asset "type" missing, assuming default type=application/rasterio:
{'name': 'Download HLS.S30.T13TEF.2020262T174919.v1.5.SAA.tif', 'href': 'https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020262T174919.v1.5.SAA.tif', 'type': 'application/rasterio'}
  warnings.warn(
/opt/conda/envs/py_geo/lib/python3.8/site-packages/intake_stac/catalog.py:456: UserWarning: STAC Asset "type" missing, assuming default type=application/rasterio:
{'name': 'Download HLS.S30.T13TEF.2020262T174919.v1

application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio
application/rasterio


<xarray.DataArray (band: 1, y: 3660, x: 3660)>
dask.array<open_rasterio-29dd83288b98292f4f54817e9acf8148<this-array>, shape=(1, 3660, 3660), dtype=int16, chunksize=(1, 3660, 3660), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 4.6e+06 4.6e+06 4.6e+06 ... 4.49e+06 4.49e+06 4.49e+06
  * x        (x) float64 5e+05 5e+05 5.001e+05 ... 6.097e+05 6.097e+05 6.098e+05
Attributes:
    transform:                                         (30.0, 0.0, 499980.0, ...
    crs:                                               +init=epsg:32613
    res:                                               (30.0, 30.0)
    is_tiled:                                          1
    nodatavals:                                        (-9999.0,)
    scales:                                            (0.0001,)
    offsets:                                           (0.0,)
    descriptions:                                      ('Coastal_Aerosol',)
    ACCODE:                                            LaSRC
    add_offset:                                        0.0
    AREA_OR_POINT:                                     Area
    arop_ave_xshift(meters):                           0
    arop_ave_yshift(meters):                           0
    arop_ncp:                                          0
    arop_rmse(meters):                                 0
    arop_s2_refimg:                                    NONE
    cloud_coverage:                                    0
    DATASTRIP_ID:                                      S2B_OPER_MSI_L1C_DS_VG...
    HLS_PROCESSING_TIME:                               2020-10-02T02:22:55Z
    HORIZONTAL_CS_CODE:                                EPSG:32613
    HORIZONTAL_CS_NAME:                                WGS84 / UTM zone 13N
    L1C_IMAGE_QUALITY:                                 NONE
    L1_PROCESSING_TIME:                                2020-09-18T20:43:45.02...
    long_name:                                         Coastal_Aerosol
    MEAN_SUN_AZIMUTH_ANGLE:                            161.216344619847
    MEAN_SUN_ZENITH_ANGLE:                             41.0986354424001
    MEAN_VIEW_AZIMUTH_ANGLE:                           284.375726756822
    MEAN_VIEW_ZENITH_ANGLE:                            8.11793848957899
    MSI band 01 bandpass adjustment slope and offset:  0.995900, -0.000200
    MSI band 02 bandpass adjustment slope and offset:  0.977800, -0.004000
    MSI band 03 bandpass adjustment slope and offset:  1.007500, -0.000800
    MSI band 04 bandpass adjustment slope and offset:  0.976100, 0.001000
    MSI band 11 bandpass adjustment slope and offset:  1.000000, -0.000300
    MSI band 12 bandpass adjustment slope and offset:  0.986700, 0.000400
    MSI band 8a bandpass adjustment slope and offset:  0.996600, 0.000000
    NBAR_SOLAR_ZENITH:                                 42.1699135122426
    NCOLS:                                             3660
    NROWS:                                             3660
    OVR_RESAMPLING_ALG:                                NEAREST
    PROCESSING_BASELINE:                               02.09
    PRODUCT_URI:                                       S2B_MSIL1C_20200918T17...
    scale_factor:                                      0.0001
    SENSING_TIME:                                      2020-09-18T18:02:38.63...
    SPACECRAFT_NAME:                                   Sentinel-2B
    spatial_coverage:                                  88
    SPATIAL_RESOLUTION:                                30
    TILE_ID:                                           S2B_OPER_MSI_L1C_TL_VG...
    ULX:                                               499980
    ULY:                                               4600020
    _FillValue:                                        -9999

In [20]:
with rasterio.open(cc.B01.urlpath) as src:
    print(src.profile)

RasterioIOError: '/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020289T174319.v1.5.B01.tif' not recognized as a supported file format.

In [11]:
f = '/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020289T174319.v1.5.B01.tif'

In [12]:
xr.open_rasterio('/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020289T174319.v1.5.B01.tif')

RasterioIOError: '/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020289T174319.v1.5.B01.tif' not recognized as a supported file format.

In [13]:
import rasterio

In [14]:
with rasterio.open(f) as src:
    print(src.profile)

RasterioIOError: '/vsicurl/https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T13TEF.2020289T174319.v1.5.B01.tif' not recognized as a supported file format.

In [13]:
import gdal

In [18]:
# GDAL configurations used to successfully access LP DAAC Cloud Assets via vsicurl 
gdal.SetConfigOption("GDAL_HTTP_UNSAFESSL", "YES")
gdal.SetConfigOption('GDAL_HTTP_COOKIEFILE','~/cookies.txt')
gdal.SetConfigOption('GDAL_HTTP_COOKIEJAR', '~/cookies.txt')
gdal.SetConfigOption('GDAL_DISABLE_READDIR_ON_OPEN','YES')
gdal.SetConfigOption('CPL_VSIL_CURL_ALLOWED_EXTENSIONS','TIF')
gdal.SetConfigOption('GDAL_SKIP','DODS')

In [6]:
with rasterio.open('https://lpdaac.earthdata.nasa.gov/lp-prod-protected/HLSS30.015/HLS.S30.T10TEK.2020190T184919.v1.5.B08.tif') as src:
    print(src.profile)

{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -9999.0, 'width': 3660, 'height': 3660, 'count': 1, 'crs': CRS.from_epsg(32610), 'transform': Affine(30.0, 0.0, 499980.0,
       0.0, -30.0, 4500000.0), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'deflate', 'interleave': 'band'}


In [5]:
import rasterio